In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd '/content/drive/MyDrive/AI_writer'

/content/drive/MyDrive/AI_writer


In [3]:
!pip install underthesea
!pip install transformers
!pip install faiss-cpu 
!pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Check gpu info

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jun 21 07:20:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# import zipfile

# # path = '/content/drive/MyDrive/AI_writer/Marketing_PhoBERT_ver5.zip'
# path = '/content/drive/MyDrive/AI_writer/Marketing_Tokenizer_ver3.zip'
# with zipfile.ZipFile(path, 'r') as zip_ref:
#     zip_ref.extractall('/content/drive/MyDrive/AI_writer/models/tokenizer')


In [6]:
import pandas as pd 
import numpy as np
import os

import sys
sys.path.append('/content/drive/MyDrive/Seg3D/KiTS2019/Source')

from repos.utils import multiprocess
from repos.utils import get_progress 
from pyvi.ViTokenizer import tokenize

import multiprocessing
multiprocessing.cpu_count()


4

In [7]:
LAST_HIDDEN_STATE = 768
MAX_LENGTH = 128 


In [8]:
from repos import utils

cuda = utils.set_gpu(0)


In [9]:
data = pd.read_csv('/content/drive/MyDrive/AI_writer/[ProductLabeled]MarketingContent_ver5_4.csv', index_col=0)
data.head(5) 


,content,product_label
index,,
0,"Với bạn, nhà là. Là nơi để trở về sau những că...",others
1,"Chill nhạc chất, an toàn là nhất. LG mong muốn...",thiết bị âm thanh
2,"CUỘC SỐNG CHUNG CƯ, TRĂM NIỀM TÂM SỰ. Sống tại...",máy giặt
3,"ĐỒ CŨ HÓA MỚI, VUI CHÀO THU SANG. Thời tiết sa...",máy giặt
4,NGĂN BẢO QUẢN SỮA CHO MẸ. GIỮ TRỌN TỐT LÀNH TR...,tủ lạnh


In [10]:
# string = 'Asus dẫn đầu Intel Gen 11th'
# clean_data[clean_data['content'].apply(lambda s: s.startswith(string))]


In [11]:
data = data[~data['product_label'].isin(['others'])]
data

,content,product_label
index,,
1,"Chill nhạc chất, an toàn là nhất. LG mong muốn...",thiết bị âm thanh
2,"CUỘC SỐNG CHUNG CƯ, TRĂM NIỀM TÂM SỰ. Sống tại...",máy giặt
3,"ĐỒ CŨ HÓA MỚI, VUI CHÀO THU SANG. Thời tiết sa...",máy giặt
4,NGĂN BẢO QUẢN SỮA CHO MẸ. GIỮ TRỌN TỐT LÀNH TR...,tủ lạnh
5,Toàn cảnh chiếc màn hình được yêu thích nhất t...,tivi
...,...,...
32158,Dậy sớm đi bầu nào các bạn ơi! Diện xì tai nào...,bank
32159,Chào mừng bạn đến lớp học của CAKE!! Để xây dự...,bank
32160,Mỹ phẩm đã sản xuất theo lít thì Cake cũng bon...,bank


Prompt

In [12]:
keywords = ['pin trâu, hiệu năng khủng',
        'khả năng gập, màn hình sắc nét'
]


keyword_base = pd.DataFrame(keywords, columns=['keyword'])
keyword_base['product_label'] = 'laptop'
keyword_base


,keyword,product_label
0,"pin trâu, hiệu năng khủng",laptop
1,"khả năng gập, màn hình sắc nét",laptop


Filter product label

In [13]:
# string = 'Asus dẫn đầu Intel Gen 11th'
# knowledge_base[knowledge_base['content'].apply(lambda s: s.startswith(string))]

In [14]:
# products = data 
products = data[data['product_label'].isin(['laptop', 'điện thoại'])]

def word_segment(idc):
  row = products.iloc[idc]
  sentence = row['content']

  return tokenize(sentence)

knowledge_base = pd.DataFrame(multiprocess(word_segment, range(len(products))), columns=['content'])
knowledge_base['product_label'] = products['product_label'].values
knowledge_base 

100%|██████████| 7664/7664 [00:04<00:00, 1742.32it/s]


,content,product_label
0,"Dạo chơi chuẩn gram . Hè này , nếu không đi ch...",laptop
1,"CHẠY DEADLINE CHẲNG SỢ NGÀY MAI. Đi học , đi l...",laptop
2,"Có LG gram Nào , mình đi camp . Bạn đã sẵn_sàn...",laptop
3,LG GRAM LIỆU CÓ XỨNG_ĐÁNG ĐƯỢC MỆNH_DANH LÀ UL...,laptop
4,Người trẻ hiện_đại nên chọn Laptop như thế_nào...,laptop
...,...,...
7659,Không phải chỉ ngồi nơi văn_phòng mới có_thể l...,điện thoại
7660,Hành_trình 10 năm tiên_phong trong công_nghệ_d...,điện thoại
7661,"Cần gì 1001 app chỉnh_hình ghép ảnh phức_tạp ,...",điện thoại
7662,"Selfie 1600 tấm rồi mà vẫn chưa "" bắt "" được ""...",điện thoại


In [15]:
# string = 'Asus dẫn đầu Intel Gen 11th'
# knowledge_base[knowledge_base['content'].apply(lambda s: s.startswith(string))]

In [16]:
import numpy as np 
import torch
import torch.nn as nn

from transformers import (
    AutoTokenizer,
    AutoModel,
    PhobertTokenizer,
    RobertaTokenizer,
)

import warnings
warnings.filterwarnings('ignore')


Load model

In [17]:
tokenizer = RobertaTokenizer.from_pretrained('/content/drive/MyDrive/AI_writer/models/tokenizer/Marketing_Tokenizer_ver3')
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokens = {'input_ids': [], 'attention_mask': [] }

Embeddings 

In [18]:
def tokenize_sentences(sentence):
    update_tokens = tokenizer.encode_plus(sentence, 
                                          max_length=MAX_LENGTH,
                                          padding = 'max_length', 
                                          # padding = 'longest', 
                                          return_attention_mask=True,
                                          return_tensors='pt',
                                          truncation=True, 
                                    )

    return update_tokens

In [19]:
base_sentences  = knowledge_base['content']
query_sentences = keyword_base['keyword']


In [20]:
base_tokens = base_sentences.apply(tokenize_sentences)
query_tokens = query_sentences.apply(tokenize_sentences)

Load dataset

In [21]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from repos.training.data.base_class import StandardDataset

class PseudoDataset(StandardDataset):
    def __init__(self, msks, embedings):
        self.mask = msks
        self.embeddings = embedings

    def get_len(self):
        return len(self.embeddings)

    def get_item(self, idc):
        attn_msks = self.mask.iloc[idc]['attention_mask']
        embedding = self.embeddings.iloc[idc]['input_ids'].squeeze()

        attn_msks = self.create_msk_padding(attn_msks)

        return attn_msks, embedding

    @staticmethod
    def create_msk_padding(attn_msks):
      size = torch.Size([1, 128, LAST_HIDDEN_STATE])
      attn_msks = attn_msks.unsqueeze(-1).expand(size).squeeze()
      return attn_msks


In [22]:
from transformers import logging
logging.set_verbosity_error()

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

def generate_model(freeze_features=False, pretrained_path=None):
    # model = AutoModel.from_pretrained("bert-base-uncased")
    model = AutoModel.from_pretrained("/content/drive/MyDrive/AI_writer/models")

    if pretrained_path is not None:
        print(pretrained_path)
        cp = torch.load(pretrained_path, map_location="cpu")
        model.load_state_dict(cp['model_state_dict'])

    if freeze_features:
      print("Freeze feature")
      model = nn.Sequential(*list(model.children())[:-1])
      for p in model.parameters():
          p.requires_grad = False

    return model

model = generate_model()

In [23]:
def generate_embedding(model, tokens, init_wts=False, freeze_features=False):
    if init_wts: 
        for p in model.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
    model.eval()
    with torch.no_grad():
      tokens = tokens.cuda()
      model  = model.cuda()
      outputs = model(tokens, return_dict=True)

    embeddings = outputs.last_hidden_state
    pooler_output = outputs.pooler_output

    return embeddings, pooler_output

In [24]:
def mean_pooling(dataloader, eps=1e-9):
    def yielder():
      for idc, (attn_msks, embeddings) in get_progress(enumerate(dataloader), total=len(dataloader)):
        embeddings = embeddings.cuda()
        attn_msks  = attn_msks.cuda()

        embeddings, _   = generate_embedding(model, embeddings) # (bz, sq_len, last_hidden_state) 
        embedding_msks  = embeddings * attn_msks
        sum_embedding   = torch.sum(embedding_msks, 1)
        sum_mask        = torch.clamp(attn_msks.sum(1), min=eps)

        result          = (sum_embedding / sum_mask) 

        yield result.detach().cpu().numpy() 
    
    return list(yielder())


In [25]:
# Mean base knowledge
dataset     = PseudoDataset(msks=base_tokens, embedings=base_tokens)
dataloader  = DataLoader(dataset, batch_size=128, shuffle=True)


In [26]:
mean_base = mean_pooling(dataloader=dataloader)
mean_base = np.concatenate(mean_base)
mean_base.shape

100%|██████████| 60/60 [01:06<00:00,  1.10s/it]


(7664, 768)

In [27]:
# Mean Query Knowledge
dataset     = PseudoDataset(msks=query_tokens, embedings=query_tokens)
dataloader  = DataLoader(dataset, batch_size=128, shuffle=False)


In [28]:
mean_query = mean_pooling(dataloader=dataloader)
mean_query = np.concatenate(mean_query)
mean_query.shape

100%|██████████| 1/1 [00:00<00:00, 43.25it/s]


(2, 768)

Save embeddings

In [29]:
# import json
# from pathlib import Path 
# import pickle

# with open('embeddings.json', 'w') as f:
#     json.dump(mean_base.tolist(), f)


In [30]:
# with open('embeddings.json', 'r') as f:
#     mean_base = json.load(f)


**Cosine_similarity**

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

query = np.expand_dims(mean_query[0], axis=0)
output = cosine_similarity(query, mean_base)
idc    = np.argmax(output) 

print(idc)
print("Cosine_similarity:", knowledge_base['content'].iloc[idc])


3911
Cosine_similarity: Trải nghiệm sức_mạnh đột_phá nhờ bộ vi xử_lý AMD Ryzen 3 cùng laptop Dell Vostro 3405 chính hãng . Task có nhiều thì việc gì phải căng , deadline nào chạy cũng êm_ru vì khả_năng đa_nhiệm xuất_sắc giúp dân văn_phòng xử_lý tốt các công_việc cần_thiết . Cặp bài_trùng Windows 11 + Office bản_quyền trọn đời được cài sẵn là trợ_thủ đắc_lực để nâng năng_suất công_việc lên đến mức tối_đa .


**Faiss similarity**

In [32]:
import faiss 

MAX_LENGTH = 128
LAST_HIDDEN_STATE = 768
TOP_K = 10

class RetrievalandRanking(nn.Module):
    def __init__(self, query_embedding, base_embedding, top_k):
        super().__init__()
        self.query_vec =  query_embedding
        self.lib_vec   =  base_embedding
        self.top_k = top_k

    def check_similarity(self):
        self.index = faiss.IndexFlatL2(LAST_HIDDEN_STATE)   
        self.index.add(self.lib_vec)                             
        dis, idc = self.index.search(self.query_vec, k=self.top_k)
        return dis, idc

    def forward(self, sentences: pd.DataFrame):
        top_dis, top_idc = self.check_similarity()
        top_sentence = [sentences['content'].iloc[idc] for idc in top_idc[0]]
        label        = [sentences['product_label'].iloc[idc] for idc in top_idc[0]]
        return top_sentence, label

    def extra_repr(self):
        return f"top_k={self.top_k}, index_trained={self.index.is_trained}, index_ntotal={self.index.ntotal}"


result = RetrievalandRanking(mean_query, mean_base, top_k=TOP_K)
top_sentences, top_labels = result(knowledge_base)


In [33]:
dataset = []

for idc in range(0, 10):
  dataset.append([top_sentences[idc], top_labels[idc]])


print(keyword_base.iloc[0])

result = pd.DataFrame(dataset, columns=['content', 'product_label'])
result

keyword          pin trâu, hiệu năng khủng
product_label                       laptop
Name: 0, dtype: object


,content,product_label
0,Trải nghiệm sức_mạnh đột_phá nhờ bộ vi xử_lý A...,laptop
1,ĐỒ_HỌA ĐỈNH_CAO ĐẾN GIẢI_TRÍ TUYỆT_VỜI . Dell ...,laptop
2,"MUA MÁY XỊN , NHẬN QUÀ SANG Từ 05 / 04 / 2021 ...",laptop
3,Hiệu_năng mạnh_mẽ với mức giá hạt_dẻ ? Bài_toá...,laptop
4,"Sẵn_sàng gập mọi giới_hạn , mở vạn tiềm_năng Đ...",điện thoại
5,Công_nghệ thay_đổi ngày_càng nhanh . Cuộc_sống...,điện thoại
6,Sự_kiện ra_mắt B9 Laptop Doanh_Nhân 14 inch Mỏ...,laptop
7,Kiên_nhẫn chờ_đợi sẽ nhận lại được những điều ...,điện thoại
8,ASUSPRO là dòng laptop được thiết_kế chuyên_bi...,laptop
9,VIVOBOOK S TÔI LÀ TÂM_ĐIỂM . Thế_hệ Gen Z năng...,laptop
